In [1]:
import sys
sys.path.append('..')

from helper_fun import *
import helper_fun
from Split_functions_classify.splits_two import *
from Split_functions_classify.splits_three import *
from Split_functions_classify.splits_ten import *

SEED = 22
seed_everything(SEED)

compounds = pd.read_csv('../COVID_MOONSHOT/compounds_filtered.csv')
compounds.head(2)

,Unnamed: 0,SMILES,CID,f_avg_IC50,r_avg_IC50,ROMol,molecular_weight,n_hba,n_hbd,logp,Ro5_fulfilled,f_avg_pIC50,r_avg_pIC50
0,35,N#Cc1cccc(NC(=O)Cc2cncc3ccccc23)c1,DAR-DIA-23aa0b97-19,26.719515,14.641091,<rdkit.Chem.rdchem.Mol object at 0x7fc319ac34c0>,287.105862,3,1,3.28768,True,4.573171,4.834427
1,36,O=C(Cc1cncc2ccccc12)Nc1ccccc1,DAR-DIA-23aa0b97-20,57.590417,45.077469,<rdkit.Chem.rdchem.Mol object at 0x7fc319ac3530>,262.110613,2,1,3.41600,True,4.239650,4.346040


In [2]:
two_split(compounds)
three_split(compounds)
ten_split(compounds).head()
compound_df = compounds.copy()
compound_df["maccs"] = compound_df["SMILES"].apply(smiles_to_fp,)

,Unnamed: 0,SMILES,CID,f_avg_IC50,r_avg_IC50,ROMol,molecular_weight,n_hba,n_hbd,logp,Ro5_fulfilled,f_avg_pIC50,r_avg_pIC50,bin_2,bin_3,bin_10
0,35,N#Cc1cccc(NC(=O)Cc2cncc3ccccc23)c1,DAR-DIA-23aa0b97-19,26.719515,14.641091,<rdkit.Chem.rdchem.Mol object at 0x7fc319ac34c0>,287.105862,3,1,3.28768,True,4.573171,4.834427,0,medium,2
1,36,O=C(Cc1cncc2ccccc12)Nc1ccccc1,DAR-DIA-23aa0b97-20,57.590417,45.077469,<rdkit.Chem.rdchem.Mol object at 0x7fc319ac3530>,262.110613,2,1,3.41600,True,4.239650,4.346040,0,medium,1
2,126,Cc1c(N)cncc1NC(=O)Cc1cccc(Cl)c1,TRY-UNI-714a760b-3,4.222340,8.201698,<rdkit.Chem.rdchem.Mol object at 0x7fc31a435d20>,275.082540,3,2,2.80682,True,5.374447,5.086096,1,medium,5
3,129,Cc1ccncc1NC(=O)Cc1cccc(Cl)c1,TRY-UNI-714a760b-6,24.581194,13.895165,<rdkit.Chem.rdchem.Mol object at 0x7fc31a435e70>,260.071641,2,1,3.22462,True,4.609397,4.857136,0,medium,2
4,135,Cc1ccncc1NC(=O)Nc1cccc(Cl)c1,TRY-UNI-714a760b-12,64.508813,45.689263,<rdkit.Chem.rdchem.Mol object at 0x7fc31a436110>,261.066890,2,2,3.68742,True,4.190381,4.340186,0,medium,1


In [4]:
# Specify models for 3 bin types
model_SVM_2 = svm.SVC(kernel="rbf", C=1, gamma=0.1, probability=True)
model_SVM_3 = svm.SVC(kernel="rbf", C=1, gamma=0.1, probability=True)
model_SVM_10 = svm.SVC(kernel="rbf", C=1, gamma=0.1, probability=True)

# Fit model on single split
# performance_measures = model_training_and_validation(model_SVM_2, "SVM", splits)
# Append SVM model
# models.append({"label": "Model_SVM", "model": model_SVM})
# # Plot roc curve
# plot_roc_curves_for_models(models, static_test_x, static_test_y);